In [1]:
import nltk
from nltk.tree import Tree
import numpy as np
import random
from IPython.display import display
import matplotlib.pyplot as plt
from nltk.treeprettyprinter import TreePrettyPrinter

from typing import List, AnyStr

## Structural Ambiguity

In [31]:
sentence1_structure1 = "\
            (S  (NP (N Simone))\
                    (VP (V caught)\
                        (NP (Det the)\
                            (N butterfly))\
                        (PP (P by)\
                            (NP (Det the)\
                                (N bush))\
                            )\
                    )\
            )"
                
tree1 = Tree.fromstring(sentence1_structure1)
print(TreePrettyPrinter(tree1).text())

In [3]:
sentence1_structure2 = "\
                    (S\
                        (VP (NP (N Simone))\
                            (VP (V caught))\
                            (NP (Det the)\
                                (N butterfly)))\
                        (PP (P by)\
                            (NP (Det the)\
                                (N bush))\
                        )\
                    )"
                                
Tree.fromstring(sentence1_structure2).pretty_print(unicodelines = True, nodedist = 1)

                          S                     
         ┌────────────────┴──────────┐           
         VP                          PP         
  ┌──────┼─────────┐             ┌───┴───┐       
  NP     VP        NP            │       NP     
  │      │     ┌───┴──────┐      │   ┌───┴───┐   
  N      V    Det         N      P  Det      N  
  │      │     │          │      │   │       │   
Simone caught the     butterfly  by the     bush



In [22]:
sentence2_structure1 = "\
    (S\
        (CP (TP (NP (Det the)\
                        (N child))\
                    (VP (V looked)\
                        (PP (P at)\
                            (NP (Det the)\
                                (N lady))))))\
            (VP (V using)\
                (NP (Det the)\
                    (Adj magnifying)\
                    (N glass)))\
    )"
    
Tree.fromstring(sentence2_structure1).pretty_print(unicodelines = True, nodedist = 1)

                              S                                         
                ┌─────────────┴──────────────────────┐                   
                CP                                   │                  
                │                                    │                   
                TP                                   │                  
     ┌──────────┴─────────┐                          │                   
     │                    VP                         │                  
     │          ┌─────────┴───┐                      │                   
     │          │             PP                     VP                 
     │          │     ┌───────┴───┐         ┌────────┴──────┐            
     NP         │     │           NP        │               NP          
 ┌───┴────┐     │     │       ┌───┴───┐     │    ┌──────────┼────────┐   
Det       N     V     P      Det      N     V   Det        Adj       N  
 │        │     │     │       │       │     │

In [24]:
sentence2_structure2 = "\
    (S\
        (VP (VP (NP (Det the)\
                       (N child))\
                    (V looked))\
            (PP (PP (P at)\
                    (NP (Det the)\
                        (N lady)))\
                (VP (V using)\
                    (NP (Det the)\
                        (Adj magnifying)\
                        (N glass)))))\
    )"
    
Tree.fromstring(sentence2_structure2).pretty_print(unicodelines = True, nodedist = 1)

                          S                                         
                          │                                          
                          VP                                        
          ┌───────────────┴─────────────┐                            
          │                             PP                          
          │               ┌─────────────┴────────┐                   
          VP              PP                     VP                 
     ┌────┴─────┐     ┌───┴───┐         ┌────────┴──────┐            
     NP         │     │       NP        │               NP          
 ┌───┴────┐     │     │   ┌───┴───┐     │    ┌──────────┼────────┐   
Det       N     V     P  Det      N     V   Det        Adj       N  
 │        │     │     │   │       │     │    │          │        │   
the     child looked  at the     lady using the     magnifying glass



## Recognizer

In [467]:
def check_exist(grammar,a1:str,b1:str):
    a = nltk.grammar.Nonterminal(a1)
    b = nltk.grammar.Nonterminal(b1)
    rules = grammar.productions()
    choices = [rule for rule in rules if rule.rhs() == (a,b)]
    print(choices)
# print(choices)

In [45]:
grammar = nltk.data.load("../data/atis-grammar-cnf.cfg")
#run once
nltk.download("large_grammars")
sentences = nltk.data.load("grammars/large_grammars/atis_sentences.txt", "auto")
print(sentences)

In [48]:
test_sentences = nltk.parse.util.extract_test_sentences(sentences)
print(test_sentences)

[(['i', 'need', 'a', 'flight', 'from', 'charlotte', 'to', 'las', 'vegas', 'that', 'makes', 'a', 'stop', 'in', 'saint', 'louis', '.'], 2085), (['what', 'is', 'the', 'cheapest', 'one', 'way', 'flight', 'from', 'phoenix', 'to', 'san', 'diego', 'that', 'arrives', 'in', 'the', 'morning', 'on', 'thursday', 'june', 'second', '.'], 1380), (['what', 'is', 'the', 'cheapest', 'one', 'way', 'flight', 'from', 'columbus', 'to', 'indianapolis', '.'], 50), (['is', 'there', 'a', 'flight', 'from', 'memphis', 'to', 'los', 'angeles', '.'], 18), (['what', 'aircraft', 'is', 'this', '.'], 0), (['please', 'show', 'me', 'the', 'flights', 'from', 'chicago', 'to', 'detroit', 'that', 'arrive', 'at', 'six', 'p.m.', 'next', 'tuesday', '.'], 20), (['what', 'flights', 'are', 'available', 'between', 'chicago', 'and', 'indianapolis', 'next', 'wednesday', 'between', 'eleven', 'a.m.', 'and', 'one', 'p.m', '.'], 0), (['please', 'book', 'a', 'one', 'way', 'coach', 'fare', 'from', 'chicago', 'to', 'indianapolis', 'on', 'uni

In [49]:
len(test_sentences)

98

In [57]:
no_parse = [test_sentences[index] for index in range(len(test_sentences)) if test_sentences[index][1] == 0]
len(no_parse)

28

In [511]:
sentence, value = test_sentences[3]
print(sentence , value)

['is', 'there', 'a', 'flight', 'from', 'memphis', 'to', 'los', 'angeles', '.'] 18


In [512]:
type(grammar.productions(rhs = "los")[0].lhs())

nltk.grammar.Nonterminal

In [513]:
for word in sentence:
    print(word)
    print(grammar.productions(rhs = word))

is
[VERB_BEZ -> 'is', pt_verb_bez -> 'is']
there
[there -> 'there', ADV_RB -> 'there', AVP_RB -> 'there']
a
[NOUN_NP -> 'a', ADJ_AT -> 'a', NP_NP -> 'a', a -> 'a', SIGMA -> 'a', AVPNP_NP -> 'a', PREP_IN -> 'a', NAPPOS_NP -> 'a']
flight
[VP_VB -> 'flight', NOUN_NN -> 'flight', INFCL_VB -> 'flight', NP_NN -> 'flight', SIGMA -> 'flight', AVPNP_NN -> 'flight', flight -> 'flight', VERB_VB -> 'flight']
from
[PREP_IN -> 'from', pt_prep_in -> 'from']
memphis
[NOUN_NP -> 'memphis', NP_NP -> 'memphis', SIGMA -> 'memphis', memphis -> 'memphis', AVPNP_NP -> 'memphis', NAPPOS_NP -> 'memphis']
to
[ADV_RB -> 'to', PREP_IN -> 'to', AVP_RB -> 'to', to -> 'to']
los
[los -> 'los']
angeles
[angeles -> 'angeles']
.
[pt_char_per -> '.']


In [245]:
possible_lhs = [rule.lhs() for rule in grammar.productions(rhs = "louis")]
# right = 
for left in possible_lhs:
    rules = grammar.productions(rhs = left)
    [rule for rule in rules if rule.rhs() == (left,)]

[NOUN_NP, NP_NP, SIGMA, AVPNP_NP, louis, NAPPOS_NP]

In [311]:
sentence_length = len(sentence)
print("Sentence Length :",sentence_length)
CKY_matrix = [["" for x in range(sentence_length)] for x in range(sentence_length)]

# arange words along the diagonal
for index, word in enumerate(sentence):
    CKY_matrix[index][index] = word
#     word_rules = grammar.productions(rhs = word)
#     # if word has only one production rule then replace word w rhs
#     if len(word_rules) == 1:
#         CKY_matrix[index][index] = word_rules[0].lhs()
    
for index, word in zip(range(sentence_length-1,-1,-1),sentence[::-1]):
    if CKY_matrix[index][index] == word:
        word_rules = grammar.productions(rhs = word)
        possible_lhs = [rule.lhs() for rule in word_rules]
        
        # save_val = 0
        # right = CKY_matrix[index+1][index+1]
        # for left in possible_lhs:
        #     rules = grammar.productions(rhs = left)
        #     val = len([rule for rule in rules if rule.rhs() == (left,right)])
        #     if val > save_val:
        #         save_val = val
        #         save_left = left
        
        # if save_val == 0:
        #     for left in possible_lhs:
        #         rules = grammar.productions(rhs = left)
        #         val = len(rules)
        #         if val > save_val:
        #             save_val = val
        #             save_left = left
            
        CKY_matrix[index][index] = possible_lhs
    

Sentence Length : 17


In [312]:
CKY_matrix

[[[i, SIGMA, PRON_PPSS, NP_PPSS],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  [pt_verb_md, VERB_MD],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  [NOUN_NP, ADJ_AT, NP_NP, a, SIGMA, AVPNP_NP, PREP_IN, NAPPOS_NP],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  [VP_VB, NOUN_NN, INFCL_VB, NP_NN, SIGMA, AVPNP_NN, flight, VERB_VB],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  [PREP_IN, pt_prep_in],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  [NOUN_NP, NP_NP, SIGMA, charlotte, AVPNP_NP, NAPPOS_NP],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  [ADV_RB, PREP_IN, AVP_RB, to],
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['',


In [419]:
# Method 1

sentence_length = len(sentence)
print("Sentence Length :",sentence_length)
CKY_matrix = [["" for x in range(sentence_length+1)] for x in range(sentence_length)]

all_rules = grammar.productions()

for i in range(0,sentence_length):
    word_rules = grammar.productions(rhs = sentence[i])
    CKY_matrix[i][i+1] = [rule.lhs() for rule in word_rules]

# CKY_matrix

for b in range(2,sentence_length):
    # print(b)
    for i in range(1,sentence_length-b+1):
        all_lhs = []
        for k in range(1,b):
            print(b,i,k)
            B = CKY_matrix[i][i+k]
            C = CKY_matrix[i+k][i+b]
            # print(B)
            
            for x in B:
                for y in C:
                    # print(x,y)
                    lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
                    lefts = list(set(lefts))
                    # print(lefts)
                    for left in lefts:
                        if left not in all_lhs:
                            all_lhs.append(left)
                            
            print(all_lhs)
        CKY_matrix[i][i+b] = all_lhs
    
# CKY_matrix

Sentence Length : 17
2 1 1
[]
2 2 1
[NP_NP, SIGMA, CJJ, KLA, KKW, ISN, IIN, NAPPOS_NN, JXX, KIF, NP_NN, JGM, AVPNP_NN, JGN, IKV, FYR, KJA, JUT, JXK, INB, JVM, JVV, IKR, FMM, IKD, JVB, KNH, RELCL_VB, JDV, KIE, PP_NN]
2 3 1
[HIH, FNB, JUE, SIGMA, JUZ, JVH, NP_NN, JXT, FNH, IZB, JLS, IEV, KPU]
2 4 1
[PP_NP, KKO]
2 5 1
[NP_NP, SIGMA, KLL, KKY, FSS]
2 6 1
[KLX, PP_NPS]
2 7 1
[NP_NPS, SIGMA, KMI, NP_NP, NOUN_NP, NAPPOS_NP, JQU, AVPNP_NP]
2 8 1
[]
2 9 1
[RELCL_VBZ]
2 10 1
[KRC, VP_VBZ, KQS]
2 11 1
[NP_NP, SIGMA, CJJ, KLA, KKW, ISN, IIN, NAPPOS_NN, JXX, KIF, NP_NN, JGM, AVPNP_NN, JGN, IKV, FYR, KJA, JUT, JXK, INB, JVM, JVV, IKR, FMM, IKD, JVB, KNH, RELCL_VB, JDV, KIE, PP_NN]
2 12 1
[HIH, FNB, JUE, SIGMA, JUZ, JVH, NP_NN, JXT, FNH, IZB, JLS, IEV, KPU]
2 13 1
[PP_NP, KKO]
2 14 1
[KKX, NP_NP, SIGMA, KEF, KLD, KLU, NAPPOS_NP, IRP, KSP, JPM, NOUN_NP, AVPNP_NP, JCX, JHR, KQI, VP_VB, JLC, JKT, KQD]
2 15 1
[IDF, NP_NP, SIGMA, KFD, IDJ, KEN, KEO, BKX, DMU, GMB, BMH, GRR, JJE, JII, GHZ, AWH, DEP, BDN, G

In [420]:
# # Method 2

# sentence_length = len(sentence)
# print("Sentence Length :",sentence_length)
# CKY_matrix2 = [["" for x in range(sentence_length+1)] for x in range(sentence_length)]

# all_rules = grammar.productions()

# for i in range(0,sentence_length):
#     word_rules = grammar.productions(rhs = sentence[i])
#     CKY_matrix2[i][i+1] = [rule.lhs() for rule in word_rules]

# # CKY_matrix

# for b in range(2,sentence_length):
#     # print(b)
#     for i in range(1,sentence_length-b+1):
#         # all_lhs = []
#         CKY_matrix2[i][i+b] = []
#         for k in range(1,b):
#             print(b,i,k)
#             B = CKY_matrix2[i][i+k]
#             C = CKY_matrix2[i+k][i+b]
#             # print(B)
#             # all_lhs = []
            
#             for x in B:
#                 for y in C:
#                     # print(x,y)
#                     lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
#                     lefts = list(set(lefts))
#                     # print(lefts)
#                     for left in lefts:
#                         if left not in CKY_matrix2[i][i+b]:
#                             CKY_matrix2[i][i+b].append(left)
                            
#             # print(all_lhs)
#         # CKY_matrix[i][i+b] = all_lhs
    
# # CKY_matrix

Sentence Length : 17
2 1 1
2 2 1
2 3 1
2 4 1
2 5 1
2 6 1
2 7 1
2 8 1
2 9 1
2 10 1
2 11 1
2 12 1
2 13 1
2 14 1
2 15 1
3 1 1
3 1 2
3 2 1
3 2 2
3 3 1
3 3 2
3 4 1
3 4 2
3 5 1
3 5 2
3 6 1
3 6 2
3 7 1
3 7 2
3 8 1
3 8 2
3 9 1
3 9 2
3 10 1
3 10 2
3 11 1
3 11 2
3 12 1
3 12 2
3 13 1
3 13 2
3 14 1
3 14 2
4 1 1
4 1 2
4 1 3
4 2 1
4 2 2
4 2 3
4 3 1
4 3 2
4 3 3
4 4 1
4 4 2
4 4 3
4 5 1
4 5 2
4 5 3
4 6 1
4 6 2
4 6 3
4 7 1
4 7 2
4 7 3
4 8 1
4 8 2
4 8 3
4 9 1
4 9 2
4 9 3
4 10 1
4 10 2
4 10 3
4 11 1
4 11 2
4 11 3
4 12 1
4 12 2
4 12 3
4 13 1
4 13 2
4 13 3
5 1 1
5 1 2
5 1 3
5 1 4
5 2 1
5 2 2
5 2 3
5 2 4
5 3 1
5 3 2
5 3 3
5 3 4
5 4 1
5 4 2
5 4 3
5 4 4
5 5 1
5 5 2
5 5 3
5 5 4
5 6 1
5 6 2
5 6 3
5 6 4
5 7 1
5 7 2
5 7 3
5 7 4
5 8 1
5 8 2
5 8 3
5 8 4
5 9 1
5 9 2
5 9 3
5 9 4
5 10 1
5 10 2
5 10 3
5 10 4
5 11 1
5 11 2
5 11 3
5 11 4
5 12 1
5 12 2
5 12 3
5 12 4
6 1 1
6 1 2
6 1 3
6 1 4
6 1 5
6 2 1
6 2 2
6 2 3
6 2 4
6 2 5
6 3 1
6 3 2
6 3 3
6 3 4
6 3 5
6 4 1
6 4 2
6 4 3
6 4 4
6 4 5
6 5 1
6 5 2
6 5 3
6 5 4
6 5 5
6 6 1
6 6

In [462]:
for b in range(1,sentence_length+1):
    for i in range(0,sentence_length-b+1):
        # all_lhs = []
        for k in range(0,b):
            print(b,i,k)
    #         B = CKY_matrix[i][i+k]
    #         C = CKY_matrix[i+k][i+b]
    #         print(B)
    #         print(C)
    # if b == 2:
    #     break

1 0 0
1 1 0
1 2 0
1 3 0
1 4 0
1 5 0
1 6 0
1 7 0
1 8 0
1 9 0
1 10 0
1 11 0
1 12 0
1 13 0
1 14 0
1 15 0
1 16 0
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
2 3 0
2 3 1
2 4 0
2 4 1
2 5 0
2 5 1
2 6 0
2 6 1
2 7 0
2 7 1
2 8 0
2 8 1
2 9 0
2 9 1
2 10 0
2 10 1
2 11 0
2 11 1
2 12 0
2 12 1
2 13 0
2 13 1
2 14 0
2 14 1
2 15 0
2 15 1
3 0 0
3 0 1
3 0 2
3 1 0
3 1 1
3 1 2
3 2 0
3 2 1
3 2 2
3 3 0
3 3 1
3 3 2
3 4 0
3 4 1
3 4 2
3 5 0
3 5 1
3 5 2
3 6 0
3 6 1
3 6 2
3 7 0
3 7 1
3 7 2
3 8 0
3 8 1
3 8 2
3 9 0
3 9 1
3 9 2
3 10 0
3 10 1
3 10 2
3 11 0
3 11 1
3 11 2
3 12 0
3 12 1
3 12 2
3 13 0
3 13 1
3 13 2
3 14 0
3 14 1
3 14 2
4 0 0
4 0 1
4 0 2
4 0 3
4 1 0
4 1 1
4 1 2
4 1 3
4 2 0
4 2 1
4 2 2
4 2 3
4 3 0
4 3 1
4 3 2
4 3 3
4 4 0
4 4 1
4 4 2
4 4 3
4 5 0
4 5 1
4 5 2
4 5 3
4 6 0
4 6 1
4 6 2
4 6 3
4 7 0
4 7 1
4 7 2
4 7 3
4 8 0
4 8 1
4 8 2
4 8 3
4 9 0
4 9 1
4 9 2
4 9 3
4 10 0
4 10 1
4 10 2
4 10 3
4 11 0
4 11 1
4 11 2
4 11 3
4 12 0
4 12 1
4 12 2
4 12 3
4 13 0
4 13 1
4 13 2
4 13 3
5 0 0
5 0 1
5 0 2
5 0 3
5 0 4
5 1 0
5 1 1
5 1 2
5 

In [465]:
CKY_matrix[0]

['',
 [i, SIGMA, PRON_PPSS, NP_PPSS],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [443]:
b, i ,k = 1,0,0
print("1",CKY_matrix[i][i+k])
print("2",CKY_matrix[i+k][i+b])
B = CKY_matrix[i][i+k]
C = CKY_matrix[i+k][i+b]
# print(B)
# all_lhs = []

# for x in B:
#     for y in C:
#         # print(x,y)
#         lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
#         lefts = list(set(lefts))
#         print(lefts)
#         for left in lefts:
#             if left not in all_lhs:
#                 all_lhs.append(left)

1 
2 [i, SIGMA, PRON_PPSS, NP_PPSS]


# Recognizer Method 2

In [577]:
# Method 3

print(sentence)
sentence_length = len(sentence)
print("Sentence Length :",sentence_length)
CKY_matrix3 = [["" for x in range(sentence_length+1)] for x in range(sentence_length)]

print("Number of rows : ",len(CKY_matrix3))
print("Number of columns : ",len(CKY_matrix3[:][0]))

assert CKY_matrix3[0][sentence_length] == ''

for index, word in enumerate(sentence):
    CKY_matrix3[index][index] = word
    word_rules = grammar.productions(rhs = word)
    CKY_matrix3[index][index+1] = [rule.lhs() for rule in word_rules]
    
# CKY_matrix3
all_rules = grammar.productions()

for b in range(2,sentence_length+2):
    for i in range(1,sentence_length-b+1):
        CKY_matrix3[i][i+b] = []
        for k in range(1,b):
            # print(b,i,k)
            B = CKY_matrix3[i][i+k]
            C = CKY_matrix3[i+k][i+b]
            # print(B)
            # print(C)
            # all_lhs = []
            
            for x in B:
                for y in C:
                    # print(x,y)
                    lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
                    lefts = list(set(lefts))
                    # print(lefts)
                    # print(lefts)
                    for left in lefts:
                        if left not in CKY_matrix3[i][i+b]:
                            CKY_matrix3[i][i+b].append(left)
                            
for b in range(2,sentence_length+1):
    i = 0
    CKY_matrix3[i][i+b] = []
    for k in range(1,b):
        # print(b,i,k)
        B = CKY_matrix3[i][i+k]
        C = CKY_matrix3[i+k][i+b]
        # print(B)
        # print(C)
        # all_lhs = []
        
        for x in B:
            for y in C:
                # print(x,y)
                lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
                lefts = list(set(lefts))
                # print(lefts)
                # print(lefts)
                for left in lefts:
                    if left not in CKY_matrix3[i][i+b]:
                        CKY_matrix3[i][i+b].append(left)
                        
if nltk.grammar.Nonterminal("SIGMA") in CKY_matrix3[0][sentence_length]:
    print(f"True, there is the Start symbol in index {0},{sentence_length}")
else:
    print(f"False, there is no Start symbol in index {0},{sentence_length}")

['is', 'there', 'a', 'flight', 'from', 'memphis', 'to', 'los', 'angeles', '.']
Sentence Length : 10
Number of rows :  10
Number of columns :  11
True, there is the Start symbol in index 0,10


# ---------------------------------------------------

In [417]:
all_lhs

[HIH, FNB, JUE, SIGMA, JUZ, JVH, NP_NN, JXT, FNH, IZB, JLS, IEV, KPU]

In [392]:
check_exist(grammar,"VERB_MD","NOUN_NP")

[]


In [303]:
for col in range(sentence_length-1,-1,-1):
    for row in range(sentence_length-1,-1,-1):
        print(row,col)
        if len(CKY_matrix[row][col]) != 0 :
            continue
        else:
            right = CKY_matrix[col][col]
            left = CKY_matrix[row][row]
            rules = grammar.productions()
            choices = [rule for rule in rules if rule.rhs() == (left,right)]
            if choices == []:
                continue
            else:
                save_val = 0
                for choice in choices:
                    choice_rules = grammar.productions(rhs = choice.lhs())
                    # print(choice_options)
                    val = len(choice_rules)
                    # print(val)
                    if val > save_val:
                        save_val = val
                        save_left = choice.lhs()
                if save_val == 0:
                    save_left = random.sample(choices,1)[0].lhs()
            
            CKY_matrix[row][col] = save_left
            

16 16
15 16
14 16
13 16
12 16
11 16
10 16
9 16
8 16
7 16
6 16
5 16
4 16
3 16
2 16
1 16
0 16
16 15
15 15
14 15
13 15
12 15
11 15
10 15
9 15
8 15
7 15
6 15
5 15
4 15
3 15
2 15
1 15
0 15
16 14
15 14
14 14
13 14
12 14
11 14
10 14
9 14
8 14
7 14
6 14
5 14
4 14
3 14
2 14
1 14
0 14
16 13
15 13
14 13
13 13
12 13
11 13
10 13
9 13
8 13
7 13
6 13
5 13
4 13
3 13
2 13
1 13
0 13
16 12
15 12
14 12
13 12
12 12
11 12
10 12
9 12
8 12
7 12
6 12
5 12
4 12
3 12
2 12
1 12
0 12
16 11
15 11
14 11
13 11
12 11
11 11
10 11
9 11
8 11
7 11
6 11
5 11
4 11
3 11
2 11
1 11
0 11
16 10
15 10
14 10
13 10
12 10
11 10
10 10
9 10
8 10
7 10
6 10
5 10
4 10
3 10
2 10
1 10
0 10
16 9
15 9
14 9
13 9
12 9
11 9
10 9
9 9
8 9
7 9
6 9
5 9
4 9
3 9
2 9
1 9
0 9
16 8
15 8
14 8
13 8
12 8
11 8
10 8
9 8
8 8
7 8
6 8
5 8
4 8
3 8
2 8
1 8
0 8
16 7
15 7
14 7
13 7
12 7
11 7
10 7
9 7
8 7
7 7
6 7
5 7
4 7
3 7
2 7
1 7
0 7
16 6
15 6
14 6
13 6
12 6
11 6
10 6
9 6
8 6
7 6
6 6
5 6
4 6
3 6
2 6
1 6
0 6
16 5
15 5
14 5
13 5
12 5
11 5
10 5
9 5
8 5
7 5
6 5
5 5
4

In [306]:
CKY_matrix[-1][:]

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', pt_char_per]

In [288]:
word_rules = grammar.productions(rhs = "in")
possible_lhs = [rule.lhs() for rule in word_rules]
print(possible_lhs)
save_val = 0
right = nltk.grammar.Nonterminal("VERB_VB")
for left in possible_lhs:
    print(left)
    rules = grammar.productions(rhs = left)
    print(rules)
    print([rule for rule in rules if rule.rhs() == (left,right)])
    val = len([rule for rule in rules if rule.rhs() == (left,right)])
    print(val)
    if val > save_val:
        save_val = val
        save_left = left

[PREP_IN, in]
PREP_IN
[FKE -> PREP_IN PP_CC, FID -> PREP_IN FIE, HFS -> PREP_IN HFT, HFU -> PREP_IN HFV, HBJ -> PREP_IN SUBCL_VBN, HBH -> PREP_IN PP_NP, GQG -> PREP_IN GQH, IHQ -> PREP_IN IHR, IHS -> PREP_IN IHT, IIM -> PREP_IN IIN, PF -> PREP_IN PG, PG -> PREP_IN pt_char_per, JIZ -> PREP_IN pt_char_per, PP_PPSS -> PREP_IN KMK, JFY -> PREP_IN PP_NNS, JFX -> PREP_IN PP_CD, JCP -> PREP_IN PP_NP, JAV -> PREP_IN PP_NP, IXO -> PREP_IN IXP, IRO -> PREP_IN IRP, IMPR_VB -> PREP_IN DZZ, IMPR_VB -> PREP_IN EAC, BAP -> PREP_IN pt_char_per, BAD -> PREP_IN pt_char_per, BBF -> PREP_IN pt_char_per, BBI -> PREP_IN BBJ, BLB -> PREP_IN pt_char_per, BLJ -> PREP_IN pt_char_per, BLF -> PREP_IN pt_char_per, PP_VBZ -> PREP_IN FSB, PP_VBG -> PREP_IN COA, PP_VBG -> PREP_IN COE, PP_VBG -> PREP_IN FRM, PP_VBG -> PREP_IN FRP, PP_VBG -> PREP_IN FRS, PP_VBG -> PREP_IN FRV, PP_VBG -> PREP_IN FRY, PP_VBG -> PREP_IN IUY, PP_VBG -> PREP_IN IVA, PP_VBG -> PREP_IN IVC, PP_VBG -> PREP_IN IVE, PP_VBG -> PREP_IN IVG, PP_VBG

In [302]:
a = nltk.grammar.Nonterminal("VERB_VB")
b = nltk.grammar.Nonterminal("NP_NP")
rules = grammar.productions()
choices = [rule for rule in rules if rule.rhs() == (a,b)]
# print(choices)
save_val = 0
for choice in choices:
    choice_options = grammar.productions(rhs = choice.lhs())
    # print(choice_options)
    val = len(choice_options)
    # print(val)
    if val > save_val:
        save_val = val
        save_left = choice.lhs()
if save_val == 0:
    save_left = random.sample(choices,1)[0].lhs()

[JLC -> VERB_VB NP_NP, JHR -> VERB_VB NP_NP, JCX -> VERB_VB NP_NP, VP_VB -> VERB_VB NP_NP, KQI -> VERB_VB NP_NP]
[]
0
[]
0
[]
0
[GGL -> VP_VB pt_char_per, GGJ -> VP_VB pt_char_per, GGV -> VP_VB pt_char_per, GGR -> VP_VB pt_char_per, GHP -> VP_VB pt_char_per, GHJ -> VP_VB pt_char_per, FTD -> VP_VB FTE, FTA -> VP_VB FTB, FTM -> VP_VB FTN, FTJ -> VP_VB FTK, FWM -> VP_VB FWN, FWP -> VP_VB FWQ, GTN -> VP_VB pt_char_per, GZG -> VP_VB GZH, GZE -> VP_VB GZF, IEM -> VP_VB IEN, JJI -> VP_VB pt_char_per, IYI -> VP_VB IYJ, IYM -> VP_VB IYN, IYO -> VP_VB IYP, IYG -> VP_VB IYH, IMPR_CC -> VP_VB GTM, CYJ -> VP_VB pt_char_per, CYH -> VP_VB CYI, CYG -> VP_VB pt_char_per, CXX -> VP_VB pt_char_per, CXY -> VP_VB CXZ, CYE -> VP_VB CYF, CYD -> VP_VB pt_char_per, CYA -> VP_VB pt_char_per, CYB -> VP_VB CYC, CXJ -> VP_VB CXK, CXM -> VP_VB CXN, CXS -> VP_VB CXT, CXU -> VP_VB pt_char_per, CXO -> VP_VB pt_char_per, CWZ -> VP_VB pt_char_per, CXC -> VP_VB pt_char_per, CXA -> VP_VB CXB, DXW -> VP_VB pt_char_per, DXU

In [300]:
save_left

VP_VB

In [257]:
CKY_matrix[15][15]

'louis'

In [ ]:
row = 15
col = 15
possible_lhs = [rule.lhs() for rule in grammar.productions(rhs = "louis")]
# right = 
for left in possible_lhs:
    rules = grammar.productions(rhs = left)
    [rule for rule in rules if rule.rhs() == (left,)]

In [161]:
CKY_matrix[-5:]

[['', '', '', '', '', '', '', '', '', '', '', '', NOUN_NN, 'in', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', in, 'saint', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', SIGMA, 'louis', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', NP_NP, '.'],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', pt_char_per]]

In [200]:
CKY_matrix[16][15]

NP_NP

In [202]:
rules = grammar.productions(rhs = CKY_matrix[16][15])
rules

[GCV -> NP_NP pt_char_per,
 GDT -> NP_NP pt_char_per,
 GED -> NP_NP pt_char_per,
 GFP -> NP_NP pt_char_per,
 GGH -> NP_NP pt_char_per,
 GHZ -> NP_NP pt_char_per,
 GKB -> NP_NP pt_char_per,
 NAPPOS_CC -> NP_NP JLW,
 GBD -> NP_NP pt_char_per,
 FRM -> NP_NP FRN,
 FRP -> NP_NP FRQ,
 FRS -> NP_NP FRT,
 FRV -> NP_NP FRW,
 FSK -> NP_NP FSL,
 FQU -> NP_NP FQV,
 FQX -> NP_NP FQY,
 FRD -> NP_NP FRE,
 FRA -> NP_NP FRB,
 FRE -> NP_NP FRF,
 FTY -> NP_NP FTZ,
 FUE -> NP_NP FUF,
 FUB -> NP_NP FUC,
 FYC -> NP_NP FYD,
 FYR -> NP_NP NOUN_NN,
 FWV -> NP_NP FWW,
 FIE -> NP_NP FIF,
 FME -> NP_NP FMF,
 FMB -> NP_NP FMC,
 FLH -> NP_NP FLI,
 FKY -> NP_NP FKZ,
 FLE -> NP_NP FLF,
 FLB -> NP_NP FLC,
 NAPPOS_NP -> NP_NP JMC,
 NAPPOS_NP -> NP_NP JMD,
 NAPPOS_NP -> NP_NP JME,
 NAPPOS_NP -> NP_NP NOUN_NP,
 NAPPOS_NN -> NP_NP JMB,
 NAPPOS_NN -> NP_NP NOUN_NN,
 FMQ -> NP_NP FMR,
 FMT -> NP_NP FMU,
 FMW -> NP_NP FMX,
 FMH -> NP_NP FMI,
 FMK -> NP_NP FML,
 FMM -> NP_NP NOUN_NN,
 FMN -> NP_NP FMO,
 FPT -> NP_NP FPU,
 FPQ

In [203]:
[rule for rule in rules if rule.rhs() == (CKY_matrix[16][15],CKY_matrix[17][16])]

[GCV -> NP_NP pt_char_per,
 GDT -> NP_NP pt_char_per,
 GED -> NP_NP pt_char_per,
 GFP -> NP_NP pt_char_per,
 GGH -> NP_NP pt_char_per,
 GHZ -> NP_NP pt_char_per,
 GKB -> NP_NP pt_char_per,
 GBD -> NP_NP pt_char_per,
 HFF -> NP_NP pt_char_per,
 GUV -> NP_NP pt_char_per,
 GXZ -> NP_NP pt_char_per,
 GOD -> NP_NP pt_char_per,
 GMR -> NP_NP pt_char_per,
 GMB -> NP_NP pt_char_per,
 GLJ -> NP_NP pt_char_per,
 GRR -> NP_NP pt_char_per,
 GQR -> NP_NP pt_char_per,
 GPF -> NP_NP pt_char_per,
 JKC -> NP_NP pt_char_per,
 JIS -> NP_NP pt_char_per,
 JII -> NP_NP pt_char_per,
 JJE -> NP_NP pt_char_per,
 BDR -> NP_NP pt_char_per,
 BDN -> NP_NP pt_char_per,
 BKX -> NP_NP pt_char_per,
 BMH -> NP_NP pt_char_per,
 CPS -> NP_NP pt_char_per,
 DWD -> NP_NP pt_char_per,
 DVO -> NP_NP pt_char_per,
 DEP -> NP_NP pt_char_per,
 DFK -> NP_NP pt_char_per,
 DMU -> NP_NP pt_char_per,
 DOW -> NP_NP pt_char_per,
 EKR -> NP_NP pt_char_per,
 JSC -> NP_NP pt_char_per,
 AWH -> NP_NP pt_char_per]

In [190]:
CKY_matrix[16][14]

''

In [235]:
CKY_matrix[-6:]

[['', '', '', '', '', '', '', '', '', '', '', PREP_IN, 'stop', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', KIE, NOUN_NN, 'in', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', in, 'saint', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', SIGMA, 'louis', ''],
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', NP_NP, '.'],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  GRR,
  pt_char_per]]

In [238]:
a = nltk.grammar.Nonterminal("in")
b = nltk.grammar.Nonterminal("NP_NP")
[rule for rule in rules if rule.rhs() == (a,b)]

[]

In [239]:
grammar.productions(rhs = "i")

[i -> 'i', SIGMA -> 'i', PRON_PPSS -> 'i', NP_PPSS -> 'i']

In [234]:
for column in range(sentence_length-1,-1,-1):
    for row in range(sentence_length,column,-1):
        print(row,column)
        if isinstance(CKY_matrix[row][column],nltk.grammar.Nonterminal):
            continue
        else:
            rules = grammar.productions(rhs = CKY_matrix[row-1][column])
            choices = [rule for rule in rules if rule.rhs() == (CKY_matrix[row-1][column],CKY_matrix[row][column+1])]
            if choices != []:
                choice = random.sample(choices,1)[0].lhs()
                print(choice)
                CKY_matrix[row][column] = choice
            else:
                continue
            # print(choices)
            # if choices != []:
            #     output_choices = choices
            #     print("hello")
            # i += 1
            
        # if i == 10:
        #     break
        # rules = grammar.productions(rhs = CKY_matrix[])
        # choices = [rule for rule in rules if rule.rhs() == (CKY_matrix[16][15],CKY_matrix[17][16])]
        # CKY_matrix[row][column] = 

17 16
17 15
16 15
17 14
16 14
15 14
17 13
16 13
15 13
14 13
17 12
16 12
15 12
14 12
13 12
17 11
16 11
15 11
14 11
13 11
12 11
17 10
16 10
15 10
14 10
13 10
12 10
11 10
17 9
16 9
15 9
14 9
13 9
12 9
11 9
10 9
17 8
16 8
15 8
14 8
13 8
12 8
11 8
10 8
9 8
17 7
16 7
15 7
14 7
13 7
12 7
11 7
10 7
9 7
8 7
17 6
16 6
15 6
14 6
13 6
12 6
11 6
10 6
9 6
8 6
7 6
17 5
16 5
15 5
14 5
13 5
12 5
11 5
10 5
9 5
8 5
7 5
6 5
17 4
16 4
15 4
14 4
13 4
12 4
11 4
10 4
9 4
8 4
7 4
6 4
5 4
17 3
16 3
15 3
14 3
13 3
12 3
11 3
10 3
9 3
8 3
7 3
6 3
5 3
4 3
17 2
16 2
15 2
14 2
13 2
12 2
11 2
10 2
9 2
8 2
7 2
6 2
5 2
4 2
3 2
17 1
16 1
15 1
14 1
13 1
12 1
11 1
10 1
9 1
8 1
7 1
6 1
5 1
4 1
3 1
2 1
17 0
16 0
15 0
14 0
13 0
12 0
11 0
10 0
9 0
8 0
7 0
6 0
5 0
4 0
3 0
2 0
1 0


In [236]:
CKY_matrix

[['i', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 [i, 'need', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['', pt_verb_md, 'a', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  '',
  NOUN_NP,
  'flight',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', '', NP_NN, 'from', '', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  '',
  '',
  '',
  PREP_IN,
  'charlotte',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', SIGMA, 'to', '', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', ADV_RB, 'las', '', '', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', SIGMA, 'vegas', '', '', '', '', '', '', '', ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  NOUN_NPS,
  'that',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', '', '', ADJ_DT, 'makes', '', '', '', '', '', ''],
 ['',
  '',
  '',
  '',
  '',
  '',
  '

In [228]:
for t in range(1,2):
    print(t)

1


In [149]:
choice = random.sample(grammar.productions(rhs = "need"),1)[0]
choice

pt_verb_md -> 'need'

In [139]:
choice.lhs()

VERB_MD

# Dealing with sentence

In [38]:
sents = "The cat is sleeping on the windowsill."

In [41]:
sents.replace("."," .").split(" ")

['The', 'cat', 'is', 'sleeping', 'on', 'the', 'windowsill', '.']

# Recognizer

In [58]:
grammar = nltk.data.load("../data/atis-grammar-cnf.cfg")
#run once
nltk.download("large_grammars")
sentences = nltk.data.load("grammars/large_grammars/atis_sentences.txt", "auto")
# print(sentences)
test_sentences = nltk.parse.util.extract_test_sentences(sentences)
# print(test_sentences)

##########################################################################################
# The test set for the ATIS grammar is a randomly selected subset of 
# the DARPA ATIS3 development test set. 
# (The full ATIS corpus is distributed by the Linguistic Data Consortium). 
#
# The original file can be found at URL:
# http://www.informatics.sussex.ac.uk/research/groups/nlp/carroll/cfg-resources/
#
# Modified for NLTK by Peter Ljunglöf <peter.ljunglof@heatherleaf.se>
# Each line begins with the number of parse trees according to the grammar.
##########################################################################################

2085 : i need a flight from charlotte to las vegas that makes a stop in saint louis .
1380 : what is the cheapest one way flight from phoenix to san diego that arrives in the morning on thursday june second .
50 : what is the cheapest one way flight from columbus to indianapolis .
18 : is there a flight from memphis to los angeles .
0 : what aircraft is th

[nltk_data] Downloading package large_grammars to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package large_grammars is already up-to-date!


In [92]:
no_parse = [test_sentences[index][0] for index in range(len(test_sentences)) if test_sentences[index][1] == 0 and len(test_sentences[index][0]) <= 9]
print(len(no_parse))
yes_parse = [test_sentences[index][0] for index in range(len(test_sentences)) if test_sentences[index][1] != 0 and len(test_sentences[index][0]) <= 7]
print(len(yes_parse))

10
18


In [93]:
random.seed(42)
grammatical = random.sample(yes_parse,10)
ungrammatical = random.sample(no_parse,10)
print(grammatical[0])
print(ungrammatical[0])

['show', 'the', 'flights', '.']
['what', 'is', 'the', 'flying', 'time', 'from', '.']


In [84]:
# from typing import List
# import nltk

def recognize(grammar: nltk.grammar.CFG, sentence: List[str]) -> bool:
    """
    Recognize whether a sentence in the language of grammar or not.

    Args:
        grammar: Grammar rule that is used to determine grammaticality of sentence.
        sentence: Input sentence that will be tested.

    Returns:
        truth_value: A bool value to determine whether if the sentence
        is in the grammar provided or not.
    """
    
    # print(f"Sentence : {''.join(sentence)}")
    sentence_length = len(sentence)
    print("Sentence Length :",sentence_length)
    cky_matrix = [["" for x in range(sentence_length+1)] for x in range(sentence_length)]

    print("Number of rows : ",len(cky_matrix))
    print("Number of columns : ",len(cky_matrix[:][0]))

    assert cky_matrix[0][sentence_length] == ''

    for index, word in enumerate(sentence):
        cky_matrix[index][index] = word
        word_rules = grammar.productions(rhs = word)
        cky_matrix[index][index+1] = [rule.lhs() for rule in word_rules]
        
    # cky_matrix
    all_rules = grammar.productions()

    for b in range(2,sentence_length+2):
        for i in range(1,sentence_length-b+1):
            cky_matrix[i][i+b] = []
            for k in range(1,b):
                # print(b,i,k)
                B = cky_matrix[i][i+k]
                C = cky_matrix[i+k][i+b]
                # print(B)
                # print(C)
                # all_lhs = []
                
                for x in B:
                    for y in C:
                        # print(x,y)
                        lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
                        lefts = list(set(lefts))
                        # print(lefts)
                        # print(lefts)
                        for left in lefts:
                            if left not in cky_matrix[i][i+b]:
                                cky_matrix[i][i+b].append(left)
                                
    for b in range(2,sentence_length+1):
        i = 0
        cky_matrix[i][i+b] = []
        for k in range(1,b):
            # print(b,i,k)
            B = cky_matrix[i][i+k]
            C = cky_matrix[i+k][i+b]
            # print(B)
            # print(C)
            # all_lhs = []
            
            for x in B:
                for y in C:
                    # print(x,y)
                    lefts = [rules.lhs() for rules in all_rules if rules.rhs() == (x,y)]
                    lefts = list(set(lefts))
                    # print(lefts)
                    # print(lefts)
                    for left in lefts:
                        if left not in cky_matrix[i][i+b]:
                            cky_matrix[i][i+b].append(left)
                            
    if nltk.grammar.Nonterminal("SIGMA") in cky_matrix[0][sentence_length]:
        # print(f"True, there is the Start symbol in index {0},{sentence_length}")
        return True
    else:
        # print(f"False, there is no Start symbol in index {0},{sentence_length}")
        return False

In [94]:
for sents in grammatical:
    # sents_inp = sents.replace("."," .").split(" ")
    # sents_inp = sents_inp.append(".")
    val = recognize(grammar, sents)
    if val:
        print(f"{sents} is in the language of CFG.")
    else:
        print(f"{sents} is not in the language of CFG.")

for sents in ungrammatical:
    val = recognize(grammar, sents)
    if val:
        print(f"{''.join(sents)} is in the language of CFG.")
    else:
        print(f"{''.join(sents)} is not in the language of CFG.")

['show', 'the', 'flights', '.']
Sentence Length : 4
Number of rows :  4
Number of columns :  5
['show', 'the', 'flights', '.'] is in the language of CFG.
['can', 'i', 'have', 'the', 'fare', '.']
Sentence Length : 6
Number of rows :  6
Number of columns :  7
['can', 'i', 'have', 'the', 'fare', '.'] is in the language of CFG.
['i', "'d", 'like', 'an', 'afternoon', 'flight', '.']
Sentence Length : 7
Number of rows :  7
Number of columns :  8
['i', "'d", 'like', 'an', 'afternoon', 'flight', '.'] is in the language of CFG.
['which', 'flights', 'use', 'a', 'large', 'plane', '.']
Sentence Length : 7
Number of rows :  7
Number of columns :  8
['which', 'flights', 'use', 'a', 'large', 'plane', '.'] is in the language of CFG.
['list', 'round', 'trips', '.']
Sentence Length : 4
Number of rows :  4
Number of columns :  5
['list', 'round', 'trips', '.'] is in the language of CFG.
['show', 'me', 'northwest', 'flights', 'to', 'detroit', '.']
Sentence Length : 7
Number of rows :  7
Number of columns :